## Dataset derived from [iWildCam 2021 - Starter Notebook](https://www.kaggle.com/nayuts/iwildcam-2021-starter-notebook#Explanation-for-metadata-file).

I created a 256x256 image dataset for prototyping. Using the MegaDetector detection results for each image, I cropped out the location of the animal. Images with no detection results were resized as is. After the cropping process, the cropped out images were indexed for easy use in learning. A summary of the process is shown below.

Use case is like this. 

<img src="https://raw.githubusercontent.com/tasotasoso/kaggle_media/main/iwildcam2021/model_image.png" width="***300***">

1. First we crop the image based on the bbox detected by MegaDetector.
2. In the training data, the correct answer labels are given as annotations, so we can use them to train the model.
3. Classify the cropped images of the test data with the trained model.
4. We choose the animal species and their counts of the image with the highest count among the images in the same image burst.

I published notebook [here](https://www.kaggle.com/nayuts/efficientnet-with-undersampling). There is no need to vote on the linked note, as the notebooks are only separated due to execution time if you like both notebook.

### Note

I created this dataset and benchmark to make it easier for newcomers to submit to the competition, but it did not provide much accuracy. Instance masks are released [Data](https://www.kaggle.com/c/iwildcam2021-fgvc8/data), so I think we may good performance if we solve this probrem using MOTS. I posted some resource for MOTS at [Multi-object tracking papers](https://www.kaggle.com/c/iwildcam2021-fgvc8/discussion/236338).

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo('K38_pZw_P9s')

---------------------------

# Preperation

In [ ]:
import glob 
import json
import os
import pickle

import cv2
import numpy as np 
import pandas as pd 
from PIL import Image, ImageFile, ImageFont, ImageDraw

In [ ]:
TRAIN_PATH = "../input/iwildcam2021-fgvc8/train/"
TEST_PATH = "../input/iwildcam2021-fgvc8/test/"
ANNOTATIONS_PATH = "../input/iwildcam2021-fgvc8/metadata/"

CROPED_TRAIN_PATH = "./croped_images_train/"
CROPED_TEST_PATH = "./croped_images_test/"

threshould = 0.9

In [ ]:
os.mkdir(CROPED_TRAIN_PATH)
os.mkdir(CROPED_TEST_PATH)

In [ ]:
with open(ANNOTATIONS_PATH + 'iwildcam2021_megadetector_results.json', encoding='utf-8') as json_file:
    megadetector_results =json.load(json_file)

In [ ]:
with open('../input/iwildcam2021-fgvc8/metadata/iwildcam2021_train_annotations.json', encoding='utf-8') as json_file:
    train_annotations =json.load(json_file)

In [ ]:
annotations = megadetector_results["images"]

In [ ]:
def get_crop_area(bbox, image_size):
    x1, y1,w_box, h_box = bbox
    ymin,xmin,ymax, xmax = y1, x1, y1 + h_box, x1 + w_box
    area = (xmin * image_size[0], ymin * image_size[1], 
            xmax * image_size[0], ymax * image_size[1])
    return area

In [ ]:
img_ids_train = []
img_idx_train = []
img_ids_test = []
img_idx_test = []

x_tot_list,x2_tot_list = [],[]

In [ ]:
def save_image(img, img_id, idx, is_train):  
    if is_train:
        img.save( CROPED_TRAIN_PATH + f"{img_id}_{idx}.jpg", format="jpeg")
        img_ids_train.append(f"{img_id}")
        img_idx_train.append(idx)
    else:
        img.save( CROPED_TEST_PATH + f"{img_id}_{idx}.jpg", format="jpeg")
        img_ids_test.append(f"{img_id}")
        img_idx_test.append(idx)

In [ ]:
def calc_x_and_x2_tot(img):
    
    img = np.array(img, dtype=np.uint8)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = np.transpose(img,(2,0,1))
            
    return (img/255.0).reshape(-1,3).mean(0), ((img/255.0)**2).reshape(-1,3).mean(0)

## main processing module↓

In [ ]:
def converet_images(annotation):

    size = (256,256)
    img_id = annotation["id"]
    is_train = True 
    
    try:
        detections = annotation["detections"]
    except:
        print(f"Passed {img_id}. There are no detection data.")
        return
    
    path_for_train = TRAIN_PATH + annotation["id"] + ".jpg"
    path_for_test = TEST_PATH + annotation["id"] + ".jpg"
    
    if os.path.exists(path_for_train):
        file_path = path_for_train
    elif os.path.exists(path_for_test):
        file_path = path_for_test
        is_train = False
    else:
        print(f"Passed {img_id}. There are no data.")
        return
    
    
    try:      
        img = Image.open(file_path)
    except:
        print(f"Passed {img_id}. Fail to open image.")
        print(f"pass {file_path}.")
        return
    
    for i, detection in enumerate(detections, 1):
        
        if detection["conf"] < threshould:
            continue

        if detection["category"] != "1":
            continue
            
        if len(detection) == 0:
            img = img.resize(size)
            save_image(img, img_id, 0, is_train)
            
            x_tot, x2_tot = calc_mean_and_var(img)
            x_tot_list.append(x_tot)
            x2_tot_list.append(x2_tot)
            
        else:
            crop_area = get_crop_area(detection["bbox"], img.size)
            img_croped = img.crop(crop_area).resize(size)
            save_image(img_croped, img_id, i, is_train)
        
            x_tot, x2_tot = calc_x_and_x2_tot(img_croped)
            x_tot_list.append(x_tot)
            x2_tot_list.append(x2_tot)

# Cropping

Crop by calling converet_images fuction.

In [ ]:
for annotation in annotations:
    converet_images(annotation)

The mean and variance for standardization.

In [ ]:
#image stats
img_avr =  np.array(x_tot_list).mean(0)
img_std =  np.sqrt(np.array(x2_tot_list).mean(0) - img_avr**2)
print('mean:',img_avr, ', std:', img_std)

## zipping

In [ ]:
!zip croped_images_train -r ./croped_images_train
!zip croped_images_test -r ./croped_images_test

In [ ]:
!rm -r ./croped_images_train
!rm -r ./croped_images_test

# Creating csv file for reference

In [ ]:
croped_train = {"id": img_ids_train, "idx":img_idx_train}
df_croped_train = pd.DataFrame(croped_train)
df_train_annotation = pd.DataFrame(train_annotations["annotations"])

In [ ]:
df_croped_train = df_croped_train.merge(
    df_train_annotation[["image_id", "category_id"]], 
    left_on='id', right_on='image_id')[["id", "idx", "category_id"]]

In [ ]:
df_croped_train.head()

In [ ]:
croped_test = {"id": img_ids_test, "idx":img_idx_test}
df_croped_test = pd.DataFrame(croped_test)

In [ ]:
df_croped_test.head()

In [ ]:
df_croped_train.to_csv("./croped_train.csv", index=False)
df_croped_test.to_csv("./croped_test.csv", index=False)